In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%cd drive/

/content/drive


In [4]:
%cd MyDrive/

/content/drive/MyDrive


In [5]:
%cd Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [6]:
# Google Drive içinde Colab Notebooks klasörünün içinde TDDI2022 diye bir klasör oluşturduktan sonra o klasöre erişiyoruz.
# Bu klasörü daha önce oluşturmadıysanız %md TDDI2022 komutu veya Google Drive Ara Yüzü ile klasörü oluşturunuz!
%cd TDDI2022/

# Klasörün içeriğinin görülmesi için
%ls

/content/drive/MyDrive/Colab Notebooks/TDDI2022
aramaProjesiKHKdoldurma.ipynb  kanunum-nlp-doc-analysis-dataset.csv
aramaProjesiVeriIsleme.ipynb   kanunum-nlp-doc-analysis-dataset.csv.gsheet


In [7]:
df = pd.read_csv('kanunum-nlp-doc-analysis-dataset.csv')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4142 entries, 0 to 4141
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              4142 non-null   int64  
 1   kategori        4142 non-null   object 
 2   baslik          4142 non-null   object 
 3   rega_no         2527 non-null   object 
 4   mukerrer_no     3642 non-null   float64
 5   rega_tarihi     2527 non-null   object 
 6   kurum           2488 non-null   object 
 7   mevzuat_no      1091 non-null   object 
 8   belge_sayi      43 non-null     object 
 9   mevzuat_tarihi  1905 non-null   object 
 10  donem           500 non-null    object 
 11  sira_no         500 non-null    object 
 12  madde_sayisi    4142 non-null   int64  
 13  data_text       4142 non-null   object 
 14  url             4142 non-null   object 
 15  kanunum_url     4142 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory usage: 517.9+ KB


In [ ]:
print(df.groupby('kategori').size())

In [ ]:
print(df.loc[df['kategori'] == 'Kanun Hükmünde Kararname'])

In [ ]:
print(df['kategori'][826:917])

In [ ]:
!pip install transformers==4.20.1

In [9]:
def kontrol(ry,ra,rg,rsayi,mno,my,ma,mg,liste,satir):
  if int(mg) < 10:
    mg = ('0'+mg)

  if int(ma) < 10:
    ma = ('0'+ma)

  if df['rega_tarihi'][satir] == (ry+'-'+ra+'-'+rg):
    liste.append('1')
  else:
    liste.append('0')

  if df['rega_no'][satir] ==rsayi:
    liste.append('1')
  else:
    liste.append('0')

  if df['mevzuat_no'][satir] == mno:
    liste.append('1')
  else:
    liste.append('0')

  if df['mevzuat_tarihi'][satir] == (my+'-'+ma+'-'+mg):
    liste.append('1')
  else:
    liste.append('0')

  return liste

In [28]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

#modeller = ["savasy/bert-base-turkish-squad","lserinol/bert-turkish-question-answering"]
modeller = "lserinol/bert-turkish-question-answering"

for satir in range(826,917):
  total = 0
  dogru = 0
  icerik = df['data_text'][satir][:123]
  if icerik.find("Mükerrer") == -1:
    icerik = df['data_text'][satir][:116]
  else:
    icerik = df['data_text'][satir][:123]
  icerik = icerik.replace('/r/n',' ')
  icerik = icerik.replace('/n/r',' ')
  icerik = icerik.replace('/n/n',' ')
  icerik = icerik.replace('/r/r',' ')
  icerik = icerik.replace('/n',' ')
  icerik = icerik.replace('/r',' ')
  #icerik = icerik.lower()
  print(icerik)
  sorular = ["Resmi Gazete Tarihi nedir?","Resmi Gazete Sayısı nedir?","Karar Sayısı Nedir?","Kararnamenin Tarihi nedir?"]

  cevaplar = []
  veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(modeller), tokenizer=AutoTokenizer.from_pretrained(modeller))
  for soru in sorular:
      sonuc = veriBulucu(question = soru, context = icerik)
      cevaplar.append(sonuc['answer'])
      #print("'",soru,"'","sorusuna verilen cevap:",sonuc['answer'])
  try:
    rg ,ra, ry = str(cevaplar[0]).split('.')
  except:
    rg ,ra, ry = str(cevaplar[0]).split('/')

  rsayi = cevaplar[1]
  khk , mno = cevaplar[2].split('/')
  try:
    mg , ma, my = str(cevaplar[3]).split('/')
  except:
    mg , ma, my = str(cevaplar[3]).split('.')
  
  liste = []
  cıktı = kontrol(ry,ra,rg,rsayi,mno,my,ma,mg,liste,satir)
  if cıktı == ['1','1','1','1']:
    dogru+=1
  total+=1
  print(satir,cıktı,cevaplar)
print("Toplam",total,"girdi,",dogru,"dogru girdi")

Resmî Gazete Tarihi: 30.06.1989 Resmî Gazete Sayısı: 20211 Mükerrer
Karar Sayısı : KHK/375
Kararnamenin Tarihi : 27/6/1989

826 ['1', '1', '1', '1'] ['30.06.1989', '20211', 'KHK/375', '27/6/1989']
Resmî Gazete Tarihi: 27.06.1995 Resmî Gazete Sayısı: 22326
Karar Sayısı : KHK/551
Kararnamenin Tarihi : 24/6/1995
72
827 ['1', '1', '1', '1'] ['27.06.1995', '22326', 'KHK/551', '24/6/1995']
Resmî Gazete Tarihi: 02.11.2011 Resmî Gazete Sayısı: 28103 Mükerrer
Karar Sayısı : KHK/662
Kararnamenin Tarihi : 11/10/2011
828 ['1', '1', '1', '1'] ['02.11.2011', '28103', 'KHK/662', '11/10/2011']
Resmî Gazete Tarihi: 02.11.2011 Resmî Gazete Sayısı: 28103 Mükerrer
Karar Sayısı : KHK/666
Kararnamenin Tarihi : 10/11/2011
829 ['1', '1', '1', '1'] ['02.11.2011', '28103', 'KHK/666', '10/11/2011']
Resmî Gazete Tarihi: 02.11.2011 Resmî Gazete Sayısı: 28103 Mükerrer
Karar Sayısı : KHK/661
Kararnamenin Tarihi : 24/10/2011
830 ['1', '1', '1', '1'] ['02.11.2011', '28103', 'KHK/661', '24/10/2011']
Resmî Gazete Tarihi

ValueError: ignored

In [32]:
print(df['data_text'][850][0:265])



Resmî Gazete Tarihi: 13.07.1993 Resmî Gazete Sayısı: 21636 Mükerrer
MALİYE VE GÜMRÜK BAKANLIĞININ TEŞKİLAT VE GÖREVLERİ HAKKINDA KANUN HÜKMÜNDE KARARNAMEDE DEĞİŞİKLİK YAPILMASINA İLİŞKİN KANUN HÜKMÜNDE KARARNAME
Karar Sayısı : KHK/484
Kararnamenin Tarihi : 2/7/1993


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
model = "lserinol/bert-turkish-question-answering"
soru = "Kanun nedir?"
veriBulucu = pipeline(task='question-answering', model=AutoModelForQuestionAnswering.from_pretrained(model), tokenizer=AutoTokenizer.from_pretrained(model))
for satir in range(100):
  icerik = df['data_text'][satir][:300]
  sonuc = veriBulucu(question = soru, context = icerik)
  yil, ay, gun = str(df['mevzuat_tarihi'][satir]).split('-')
  metin1=gun+'/'+ay+'/'+yil
  hedefIndis1 = icerik.find(metin1)
  hedefIndis2 = icerik.find("DAİR KANUN")
  if hedefIndis2 == -1:
    hedefIndis2 = icerik.find("HAKKINDA KANUN")
  print("Text = ",df['data_text'][satir][:300])
  print("mTarih =",metin1,"indexi =",hedefIndis1)
  print("Dair = ",sonuc['answer']," indexi =",hedefIndis2)
  print("Kategori:",df['kategori'][satir]," Kanun Adı: ",df['data_text'][satir][(hedefIndis1+11):hedefIndis2].replace("\n\n"," "),"DAİR KANUN",sep="")
  print("\n")